<a href="https://colab.research.google.com/github/ramachandran-ds/End-TO-END_DATA_SCIENCE_PROJECT/blob/main/END_TO_END_DATA_SCIENCE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================
# 1️⃣ Install Required Libraries
# =========================================
!pip install flask-ngrok scikit-learn pandas numpy

# =========================================
# 2️⃣ Data Collection
# =========================================
import pandas as pd
from sklearn.datasets import load_iris

# Load Iris dataset
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Show dataset info
print("Dataset Info:")
print(df.info())
print("\nDataset Description:")
print(df.describe())

# =========================================
# 3️⃣ Data Preprocessing
# =========================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features & target
X = df.drop('target', axis=1)
y = df['target']

# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# =========================================
# 4️⃣ Model Training
# =========================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize and train Logistic Regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = model.predict(X_test_scaled)
print("\nModel Accuracy:", accuracy_score(y_test, y_pred))

# =========================================
# 5️⃣ Save Model & Scaler
# =========================================
import pickle
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))

# =========================================
# 6️⃣ Flask API Deployment using Ngrok
# =========================================
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import numpy as np

# Load model and scaler again (simulate fresh API environment)
model = pickle.load(open('model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))

app = Flask(__name__)
run_with_ngrok(app)  # Automatically creates ngrok URL when running

@app.route('/')
def home():
    return "🌸 Iris Classification API is running! Send POST request to /predict"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json(force=True)

        # Validate input format
        if "features" not in data:
            return jsonify({"error": "No 'features' key found in JSON."}), 400

        features = data["features"]
        if len(features) != 4:
            return jsonify({"error": "Expected 4 feature values for Iris dataset."}), 400

        features = np.array(features).reshape(1, -1)

        # Scale and predict
        scaled = scaler.transform(features)
        pred_class = model.predict(scaled)[0]
        pred_label = iris.target_names[pred_class]

        return jsonify({
            "prediction_index": int(pred_class),
            "prediction_label": pred_label
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()
